In [ ]:
import os
from datetime import timedelta
import pickle
import sys


# -----------
sys.path.append(os.path.abspath(".."))
# from utils.inputs import *
# from utils.outputs import *

from utils import validate_inputs, load_species_params, all_historical_model_run, prediction_model_run, fflies_output_class
# from utils.outputs import fflies_output_class

import pandas as pd

sys.path.append(os.path.abspath(".."))

data_path = "data/"

In [9]:
dat = fflies_output_class(finish_date_list=[1, 2, 3])

In [11]:
def test_fun():
    return fflies_output_class(finish_date_list=[1, 2, 3])

In [12]:
foo = test_fun()
foo

fflies_output_class(finish_date_list=[1, 2, 3], figure=<Figure size 640x480 with 0 Axes>, value=None, array=<xarray.DataArray ()> Size: 8B
array(nan))

In [9]:
def run_plot_DD(
    start_dates,
    coordinates,
    target_species,
    historical_data_buffer=400,
    cache_path=os.path.join(data_path, "cache/pred_cache.pkl"),
    context_map=False,
    save_output=False,
    all_historical=None,
    force_prediction=False,
):
    """
    Generates a plot of completion dates for a given species at specified coordinates.

    Parameters:
    ----------
    start_dates : list of datetime
        List of start dates corresponding to each coordinate.
    coordinates : list of tuples (lat, lon)
        List of latitude and longitude pairs.
    target_species : str
        Name of the species to model.
    days_of_data : int
        Number of days of data to fetch - usually 180-200 is sufficient
    historical_data_buffer : int
        Number of days to buffer historical data for model run.
    cache_path : str
        Path to cache file for storing fetched data.
    context_map : bool
        If True, generates a context map for a single point.
    save_output : bool
        If True, saves the output data to a tiff file.
    Returns:
    -------
    matplotlib.figure.Figure
        A plot displaying degree-day completion for the given coordinates.

    Raises:
    ------
    ValueError:
        If the number of coordinates and dates do not match.
        If no coordinates are provided.
        If the start date is too early or the end date is in the future.
    """
    ###################
    ##Input Validation#
    ###################
    all_historical = validate_inputs(start_dates, coordinates, historical_data_buffer)

    # a bounding box if we have multiple points, a context map if we have a single point
    # loaded from .json file
    fly_params = load_species_params(target_species, data_path)

    # setup Modelvariables
    first_date = min(start_dates)
    last_date = max(start_dates) + timedelta(days=historical_data_buffer)
    n_days_data = (last_date - first_date).days

    ##########
    ##Model##
    ##########
    if all_historical:
        try:
            return all_historical_model_run(
                coordinates,
                start_dates,
                fly_params,
                n_days_data,
                cache_path,
                context_map,
            )
        except PredictionNeededError:
            all_historical = False
    if force_prediction or not all_historical:
        prediction_model_run(
            coordinates,
            start_dates,
            fly_params,
            n_days_data,
            cache_path,
            produce_plot=False,
        )
        # predict

In [10]:
output = run_plot_DD(
    [pd.to_datetime("2025-01-01")],
    [(34.63115, -117.338321)],
    "Mexfly",
    context_map=False,
    cache_path=os.path.join(data_path, "cache/pred_cache.pkl"),
)

er


In [11]:
print(output)

None


In [7]:
run_plot_DD(
    [pd.to_datetime("2002-01-01")],
    [(34.63115, -117.338321)],
    "Mexfly",
    context_map=True,
    cache_path=os.path.join(data_path, "cache/cache.pkl"),
)

AttributeError: 'str' object has no attribute 'strptime'

In [ ]:
run_plot_DD(
    [pd.to_datetime("2002-01-01")],
    [(34.63115, -117.338321)],
    "Mexfly",
)

NameError: name 'run_plot_DD' is not defined

run_plot_DD(
    [pd.to_datetime("2002-01-01")],
    [(34.63115, -117.338321)],
    "Mexfly",
    context_map=False,
)
run_plot_DD(
    [pd.to_datetime("2002-01-01")],
    [(34.63115, -117.338321)],
    "Mexfly",
    context_map=True,
)
### Why is map not plotting?

In [ ]:
# test statements
"""
run_plot_DD(
    [pd.to_datetime("2002-01-01")],
    [(34.63115, -117.338321)],
    "Mexfly",


run_plot_DD(
    [pd.to_datetime("2002-01-01"), pd.to_datetime("2002-05-01")],
    [(34.63115, -117.338321), (34.68115, -117.336321)],
    "Mexfly",
)
"""

'\nrun_plot_DD(\n    [pd.to_datetime("2002-01-01")],\n    [(34.63115, -117.338321)],\n    "Mexfly",\n\n\nrun_plot_DD(\n    [pd.to_datetime("2002-01-01"), pd.to_datetime("2002-05-01")],\n    [(34.63115, -117.338321), (34.68115, -117.336321)],\n    "Mexfly",\n)\n'

In [ ]:
import os


# Parameters
start_date = "2020-01-01"
bbox = (-117.63832099999999, -117.038321, 34.33115, 34.931149999999995)
cache_path = "cache/pred_cache.pkl"

# Ensure the cache directory exists
os.makedirs(os.path.dirname(cache_path), exist_ok=True)

# Fetch the data
ncss_data = fetch_ncss_data(start_date=start_date, bbox=bbox)

# Save the data to the cache
with open(cache_path, "wb") as cache_file:
    pickle.dump(ncss_data, cache_file)

print(f"NCSS data saved to {cache_path}")

NCSS data saved to cache/pred_cache.pkl
